In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/high-value-customers-identification/Ecommerce.csv', encoding='ISO-8859-1')
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:

data.isnull().sum()

In [ ]:
data.drop(['Unnamed: 8'], axis=1, inplace=True)

In [ ]:
data.CustomerID.nunique()

In [ ]:
data.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(15,12))
data.groupby('Country')['Quantity'].count().plot(kind='barh')
plt.show()

In [ ]:
data['Total']=data.Quantity*data.UnitPrice

In [ ]:
data['Invoice_Year']=pd.to_datetime(data.InvoiceDate).dt.year
data['Invoice_Month']=pd.to_datetime(data.InvoiceDate).dt.month
data['Invoice_Day']=pd.to_datetime(data.InvoiceDate).dt.day

In [ ]:
data

In [ ]:
per_yr_total=data.groupby('Invoice_Year')['Total'].sum()
per_yr_total.plot(kind='bar')
plt.show()

In 2017, the total sales were more than in 2016.

In [ ]:
per_yr_total

In [ ]:
per_monyr_total=data.groupby(['Invoice_Year','Invoice_Month'])['Total'].sum()
per_monyr_total.plot(kind='bar')
plt.show()

In november of 2017, the total was maximum

In [ ]:
plt.figure(figsize=(15,12))
max_per_stk=data.groupby('StockCode')['Total'].sum().sort_values(ascending=False)[:20]
max_per_stk.plot(kind='bar')
plt.show()

The stock code DOT had the max value of transaction over the two years.

In [ ]:
total_per_inv=data.groupby('InvoiceNo')['Total'].sum().sort_values(ascending=False)[:15]
total_per_inv.plot(kind='bar')
plt.show()

In [ ]:
plt.figure(figsize=(15,12))
sns.barplot('Invoice_Month','Total', data=data,hue='Invoice_Year')
plt.show()

For 2016, we have data for only two months, Nov and Dec and data for these two months are more than those of 2017.

In [ ]:
plt.figure(figsize=(20,20))
sns.barplot('Invoice_Day','Quantity', hue='Invoice_Year',data=data)
plt.show()

In [ ]:
df=data.groupby('CustomerID').agg({'Total':['mean','sum','max']})
df=pd.DataFrame(df)
df.columns=['TotalAmt','MeanAmt','MaxAmt']
df







In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler


In [ ]:
sc=MinMaxScaler()
x=df.values
x=sc.fit_transform(x)
x

In [ ]:

iner=[]
for i in range(1,11):
    km=KMeans(n_clusters=i, init='k-means++',random_state=0)
    km.fit(x)
    iner.append(km.inertia_)
    

In [ ]:
plt.plot(range(1,11), iner)
plt.xlabel('No. of clusters')
plt.show()


The probabale clusters could be 2,3, or 4

In [ ]:
import mpl_toolkits.mplot3d.axes3d as p3

In [ ]:
def kmeans_cluster(n):
    kmeans=KMeans(n_clusters=n, init='k-means++',random_state=0)
    kmeans.fit_predict(x)
    return kmeans

In [ ]:
def plot_clusters_3d(cluster):
    fig=plt.figure(figsize=(10,8))
    ax=p3.Axes3D(fig)
    label=cluster.labels_
    ax.scatter(x[:, 0], x[:, 1], x[:,2], c=label,s=50,edgecolor='k', cmap='viridis')
    ax.scatter(cluster.cluster_centers_[:, 0], cluster.cluster_centers_[:, 1],cluster.cluster_centers_[:, 2], s = 80, c = 'r',marker='*', label = 'Centroids')
    ax.set_xlabel('TotalAmt')
    ax.set_ylabel('MeanAmt')
    ax.set_zlabel('MaxAmt')    
    ax.set_title(str(np.unique(label).size)+' clusters')
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.show()
    

For number of clusters 2

In [ ]:
kmeans_2=kmeans_cluster(2)
plot_clusters_3d(kmeans_2)

If we plot a 2D graphs of the same

With 3 clusters

In [ ]:
kmeans_3=kmeans_cluster(3)
plot_clusters_3d(kmeans_3)

With 4 clusters

In [ ]:
kmeans_4=kmeans_cluster(4)
plot_clusters_3d(kmeans_4)

In [ ]:
import scipy.cluster.hierarchy as hc

In [ ]:
plt.figure(figsize=(50,12))
plt.title("Dendogram")
d=hc.dendrogram(hc.linkage(x, method='ward'))

In [ ]:
from sklearn.cluster import AgglomerativeClustering


In [ ]:
def findClusters(n_cluster):
    agg_clus=AgglomerativeClustering(n_clusters=n_cluster, affinity='euclidean', linkage='ward')
    agg_clus.fit_predict(x)
    return agg_clus

In [ ]:
def plot_aggclusters_3d(cluster):
    fig=plt.figure(figsize=(10,8))
    ax=p3.Axes3D(fig)
    label=cluster.labels_
    ax.scatter(x[:, 0], x[:, 1], x[:,2], c=label,s=50,edgecolor='k', cmap='plasma')
    ax.set_xlabel('TotalAmt')
    ax.set_ylabel('MeanAmt')
    ax.set_zlabel('MaxAmt')    
    ax.set_title(str(np.unique(label).size)+' clusters')
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.show()
    

In [ ]:
agg_2=findClusters(2)
plot_aggclusters_3d(agg_2)

In [ ]:
agg_3=findClusters(3)
plot_aggclusters_3d(agg_3)

In [ ]:
agg_4=findClusters(4)
plot_aggclusters_3d(agg_4)